In [4]:
import mysql.connector
import pandas as pd
import logging
from data.config import *

class Database:
    def __init__(self, host, user, password, database):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.connection = None
        self.connect()

    def connect(self):
        """
        Connects to the MySQL database.
        """
        try:
            self.connection = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            self.cursor = self.connection.cursor(dictionary=True)  # Fetch results as dictionaries for proper column names
        except mysql.connector.Error as err:
            logging.error(f"Error: {err}")
            raise

    def fetch_sales_data(self):
        """
        Executes the SQL query to fetch all sales-related data.
        """
        query = """
        SELECT
            soh.SalesOrderID,
            soh.OrderDate,
            soh.DueDate,
            soh.ShipDate,
            soh.Status,
            soh.OnlineOrderFlag,
            soh.SalesOrderNumber,
            soh.PurchaseOrderNumber,
            soh.SubTotal,
            soh.TaxAmt,
            soh.Freight,
            soh.TotalDue,
            soh.Comment,
            sod.ProductID,
            sod.OrderQty,
            sod.UnitPrice,
            sod.LineTotal,
            sod.UnitPriceDiscount,
            p.Name AS ProductName,
            p.ProductNumber,
            p.Color,
            p.StandardCost,
            p.ListPrice,
            psc.Name AS SubcategoryName,
            pc.Name AS CategoryName,
            st.Name AS TerritoryName,
            sm.Name AS ShipMethodName,
            c.CustomerID,
            c.AccountNumber,
            pa.AddressLine1,
            pa.City,
            pa.StateProvinceID,
            pa.PostalCode
        FROM Sales_SalesOrderHeader soh
        JOIN Sales_SalesOrderDetail sod ON soh.SalesOrderID = sod.SalesOrderID
        JOIN Production_Product p ON sod.ProductID = p.ProductID
        LEFT JOIN Production_ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
        LEFT JOIN Production_ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
        LEFT JOIN Sales_SalesTerritory st ON soh.TerritoryID = st.TerritoryID
        LEFT JOIN Purchasing_ShipMethod sm ON soh.ShipMethodID = sm.ShipMethodID
        LEFT JOIN Sales_Customer c ON soh.CustomerID = c.CustomerID
        LEFT JOIN Person_Address pa ON c.PersonID = pa.AddressID;
        """
        
        try:
            self.cursor.execute(query)
            result = self.cursor.fetchall()
            return result
        except mysql.connector.Error as err:
            logging.error(f"MySQL error: {err}")
            return []
    
    def close(self):
        """
        Closes the database connection.
        """
        if self.connection:
            self.connection.close()

db = Database(host=HOST, user=MYSQL_USER, password=MYSQL_PASSWORD, database=MYSQL_DATABASE)


sales_data = db.fetch_sales_data()

# If data was successfully fetched
if sales_data:
    # Convert to pandas DataFrame
    df = pd.DataFrame(sales_data)

    # Save DataFrame to CSV with proper column names
    df.to_csv('sales_data.csv', index=False)
    print("Data saved to sales_data.csv")
else:
    print("No data was fetched.")

# Close the database connection
db.close()


Data saved to sales_data.csv
